# 10. Energy Efficient ANN

This notebook explores energy-efficient neural network designs.

## Experiment Overview
- **Goal**: Explore energy-efficient neural network designs
- **Model**: Quantized and pruned networks
- **Features**: Energy consumption estimation, accuracy-efficiency trade-offs
- **Learning**: Understanding model efficiency and optimization

## What You'll Learn
- Model quantization techniques
- Energy consumption estimation
- Accuracy vs. efficiency trade-offs
- Efficient neural network design


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import time

# Add scripts directory to path
sys.path.append('../scripts')
from utils import load_mnist_data, get_device, set_seed

# Set random seed for reproducibility
set_seed(42)

# Get device
device = get_device()
print(f"Using device: {device}")-

# Load MNIST dataset
print("Loading MNIST dataset...")
train_loader, val_loader, test_loader = load_mnist_data(batch_size=64, test_split=0.2)

print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")


In [ ]:
# Define energy-efficient models
class EnergyEfficientMLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=64, num_classes=10):
        super(EnergyEfficientMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class QuantizedMLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=64, num_classes=10):
        super(QuantizedMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Create models
efficient_model = EnergyEfficientMLP().to(device)
quantized_model = QuantizedMLP().to(device)

print("Energy Efficient Model:")
print(efficient_model)
print(f"Parameters: {sum(p.numel() for p in efficient_model.parameters()):,}")

print("\nQuantized Model:")
print(quantized_model)
print(f"Parameters: {sum(p.numel() for p in quantized_model.parameters()):,}")

# Training function with timing
def train_model_with_timing(model, train_loader, val_loader, epochs=10, lr=0.001):
    """Train model and measure time."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    train_losses = []
    val_losses = []
    training_times = []
    
    start_time = time.time()
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        # Training
        model.train()
        train_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item()
        
        epoch_time = time.time() - epoch_start
        training_times.append(epoch_time)
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        if (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {epoch_time:.2f}s')
    
    total_time = time.time() - start_time
    return train_losses, val_losses, training_times, total_time

# Train both models
print("Training Energy Efficient Model...")
eff_train_losses, eff_val_losses, eff_times, eff_total_time = train_model_with_timing(
    efficient_model, train_loader, val_loader, epochs=10
)

print("\nTraining Quantized Model...")
quant_train_losses, quant_val_losses, quant_times, quant_total_time = train_model_with_timing(
    quantized_model, train_loader, val_loader, epochs=10
)

# Plot results
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.plot(eff_train_losses, label='Efficient Train')
plt.plot(eff_val_losses, label='Efficient Val')
plt.plot(quant_train_losses, label='Quantized Train')
plt.plot(quant_val_losses, label='Quantized Val')
plt.title('Training Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(2, 3, 2)
plt.plot(eff_times, label='Efficient')
plt.plot(quant_times, label='Quantized')
plt.title('Training Time per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Time (s)')
plt.legend()
plt.grid(True)

plt.subplot(2, 3, 3)
models = ['Efficient', 'Quantized']
total_times = [eff_total_time, quant_total_time]
plt.bar(models, total_times)
plt.title('Total Training Time')
plt.ylabel('Time (s)')
plt.grid(True)

plt.subplot(2, 3, 4)
params = [sum(p.numel() for p in efficient_model.parameters()), 
          sum(p.numel() for p in quantized_model.parameters())]
plt.bar(models, params)
plt.title('Model Parameters')
plt.ylabel('Count')
plt.grid(True)

plt.subplot(2, 3, 5)
model_sizes = [p * 4 / 1024 / 1024 for p in params]  # MB
plt.bar(models, model_sizes)
plt.title('Model Size')
plt.ylabel('Size (MB)')
plt.grid(True)

plt.subplot(2, 3, 6)
efficiency = [acc / time for acc, time in zip([max(eff_val_losses), max(quant_val_losses)], total_times)]
plt.bar(models, efficiency)
plt.title('Training Efficiency (Loss/Time)')
plt.ylabel('Efficiency')
plt.grid(True)

plt.tight_layout()
plt.savefig('../results/plots/energy_efficiency.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nEnergy Efficiency Summary:")
print(f"Efficient Model - Total Time: {eff_total_time:.2f}s, Parameters: {params[0]:,}")
print(f"Quantized Model - Total Time: {quant_total_time:.2f}s, Parameters: {params[1]:,}")
print(f"Time Reduction: {((eff_total_time - quant_total_time) / eff_total_time * 100):.1f}%")
print(f"Parameter Reduction: {((params[0] - params[1]) / params[0] * 100):.1f}%")
